# Web Scraping Homework - Mission to Mars

In [1]:
#import dependencies
from splinter import Browser
from bs4 import BeautifulSoup as bs
import pandas as pd
import requests
import pymongo
from flask import Flask, render_template, redirect
from flask_pymongo import PyMongo
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
# Setup config variables to enable Splinter interaction with browser
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 93.0.4577
Get LATEST driver version for 93.0.4577
Driver [C:\Users\jchan\.wdm\drivers\chromedriver\win32\93.0.4577.63\chromedriver.exe] found in cache


<strong> Hint:</strong> Use Splinter to navigate the sites when needed and BeautifulSoup to help find and parse out the necessary data.

In [3]:
# Create dictionary to store news
nasa_data = {}

## NASA Mars News

Scrape the [NASA Mars News Site](https://mars.nasa.gov/news/) and collect the latest News Title and Paragraph Text. Assign the text to variables that you can reference later.

In [4]:
# Visit Nasa news url through splinter module
nasa_url = "https://redplanetscience.com" 
browser.visit(nasa_url)

In [5]:
html = browser.html
# Create BeautifulSoup object; parse with 'html.parser'
soup = bs(html, 'html.parser')

In [6]:
# Get news title and news text by searching for appropriate div class 
news_title = soup.find('div', class_='content_title').text
news_p = soup.find('div', class_='article_teaser_body').text
print(news_title)
print(news_p)

HiRISE Views NASA's InSight and Curiosity on Mars
New images taken from space offer the clearest orbital glimpse yet of InSight as well as a view of Curiosity rolling along.


In [7]:
# Create dictionary to store data and save entries
scrape_nasa_news={"Title":news_title, "Paragraph":news_p}
scrape_nasa_news

{'Title': "HiRISE Views NASA's InSight and Curiosity on Mars",
 'Paragraph': 'New images taken from space offer the clearest orbital glimpse yet of InSight as well as a view of Curiosity rolling along.'}

In [8]:
# Save scraped data as a new entry in the dictionary
nasa_data ["Title"] = news_title
nasa_data["Paragraph"] = news_p

## JPL Mars Space Images - Featured Image

- Visit the url for JPL Featured Space Image [here](https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars).
- Use splinter to navigate the site and find the image url for the current Featured Mars Image and assign the url string to a variable called featured_image_url.
- Find the image url to the full size .jpg image. Make sure to save a complete url string for this image.

In [9]:
mars_url = "https://spaceimages-mars.com" 
browser.visit( mars_url)
image_html = browser.html

# Create BeautifulSoup object; parse with 'html.parser'
soup = bs(image_html, "html.parser")

In [10]:
featured_image = soup.find_all("img", class_ = "headerimage fade-in")[0]["src"]
featured_image_url = mars_url + featured_image
print(featured_image_url)

https://spaceimages-mars.comimage/featured/mars1.jpg


In [14]:
browser.quit()

## Mars Facts

Visit the Mars Facts webpage [here](https://galaxyfacts-mars.com/) and use Pandas to scrape the table containing facts about the planet.
Use Pandas to convert the data to a HTML table string.

In [ ]:
facts_url = "https://galaxyfacts-mars.com/"
facts_data = pd.read_html(facts_url)

facts_df = facts_data[0]
facts_table = facts_df.to_html(index=False)
facts_table

In [ ]:
# Check out table
facts_df[1]

## Mars Hemispheres:

Visit the USGS Astrogeology site [here](https://marshemispheres.com/) to obtain high resolution images for each of Mar's hemispheres.

You will need to click each of the links to the hemispheres in order to find the image url to the full resolution image.

Save both the image url string for the full resolution hemisphere image, and the Hemisphere title containing the hemisphere name. Use a Python dictionary to store the data using the keys img_url and title.

Append the dictionary with the image url string and the hemisphere title to a list. This list will contain one dictionary for each hemisphere.

<strong>Example:</strong> hemisphere_image_urls = [ {"title": "Valles Marineris Hemisphere", "img_url": "..."}, {"title": "Cerberus Hemisphere", "img_url": "..."}, {"title": "Schiaparelli Hemisphere", "img_url": "..."}, {"title": "Syrtis Major Hemisphere", "img_url": "..."}, ]

In [ ]:
# Visit hemisphere url through splinter module
hemisphere_url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(hemisphere_url)

In [ ]:
html = browser.html
# Create BeautifulSoup object; parse with 'html.parser'
soup = bs(html, 'html.parser')

In [ ]:
hemisphere_links = soup.find_all('div', class_='item')

In [ ]:
hemisphere_img_urls = []

for i in range(4): 
    browser.find_by_tag('h3')[i].click()
    soup = bs(browser.html, 'html.parser')
    img_title = soup.find("h2", class_="title").text
    img_url = soup.find("a", text ="Sample")["href"]
    hem_dic = {"title": img_title, "url": img_url}
    hemisphere_img_urls.append(hem_dic)
    browser.back()

hemisphere_img_urls

In [ ]:
browser.quit()

In [ ]:
# Define mars dictionary
mars_dict ={"news_title": news_title, "news_p": news_p, "mars_feature_img": mars_feature_img, 
            "facts_table": mars_feature_img, "hemisphere_img_urls":mars_feature_img}
mars_dict.head(2)